In [ ]:
import sionna as sn
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

In [ ]:
BATCH_SIZE = 1000
NUM_BITS_PER_SYMBOL = 4
CODERATE = 0.5
NUM_CODED_BITS = 1000
NUM_INFO_BITS = int(NUM_CODED_BITS * CODERATE)

In [ ]:
class AWGNModel(tf.keras.Model):
    
    def __init__(self, coding_type=False):
        super().__init__()

        self.coding_type = coding_type

        self.binary_source = sn.utils.BinarySource()

        constellation = sn.mapping.Constellation('qam', NUM_BITS_PER_SYMBOL)

        self.mapper = sn.mapping.Mapper(constellation=constellation)
        self.demapper = sn.mapping.Demapper('app', constellation=constellation)

        self.ldpc_encoder = sn.fec.ldpc.LDPC5GEncoder(NUM_INFO_BITS, NUM_CODED_BITS)
        self.ldpc_decoder = sn.fec.ldpc.LDPC5GDecoder(encoder=self.ldpc_encoder, hard_out=True)

        self.polar_encoder = sn.fec.polar.Polar5GEncoder(NUM_INFO_BITS, NUM_CODED_BITS)
        self.polar_decoder = sn.fec.polar.Polar5GDecoder(self.polar_encoder, dec_type='SC')

        self.channel = sn.channel.AWGN()
    
    @tf.function
    def __call__(self, batch_size, ebno_db):

        no = sn.utils.ebnodb2no(
            coderate=CODERATE,
            ebno_db=ebno_db,
            num_bits_per_symbol=NUM_BITS_PER_SYMBOL,
        )

        bits = self.binary_source([batch_size, NUM_INFO_BITS])

        if self.coding_type == 'ldpc':
            coded_bits = self.ldpc_encoder(bits)
        elif self.coding_type == 'polar':
            coded_bits = self.polar_encoder(bits)
        else:
            coded_bits = bits

        symbols = self.mapper(coded_bits)

        received_symbols = self.channel([symbols, no])

        llr = self.demapper([received_symbols, no])

        if self.coding_type == 'ldpc':
            bits_hat = self.ldpc_decoder(llr)
        elif self.coding_type == 'polar':
            bits_hat = self.polar_decoder(llr)
        else:
            bits_hat = llr


        return bits, bits_hat

In [ ]:
ber_plots = sn.utils.PlotBER('Canal AWGN')
EBNO_DBS = np.linspace(-5,10,16)

In [ ]:
no_coding_model = AWGNModel()

ber_plots.simulate(
    no_coding_model,
    batch_size=BATCH_SIZE,
    ebno_dbs=EBNO_DBS,
    num_target_bit_errors=1000,
    legend='Sem codificação',
    soft_estimates=True,
    max_mc_iter=100,
    show_fig=False
);

In [ ]:
ldpc_coding_model = AWGNModel('ldpc')

ber_plots.simulate(
    ldpc_coding_model,
    batch_size=BATCH_SIZE,
    ebno_dbs=EBNO_DBS,
    num_target_bit_errors=1000,
    legend='LDPC',
    soft_estimates=True,
    max_mc_iter=100,
    show_fig=False
);

In [ ]:
polar_coding_model = AWGNModel('polar')

ber_plots.simulate(
    polar_coding_model,
    batch_size=BATCH_SIZE,
    ebno_dbs=EBNO_DBS,
    num_target_bit_errors=1000,
    legend='Polar',
    soft_estimates=True,
    max_mc_iter=100,
    show_fig=False
);

In [ ]:
ber_plots()